<a href="https://colab.research.google.com/github/Talha-coder-01/AI-ML-Projects/blob/main/Simple_Movie_Recommender_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.impute import SimpleImputer


In [2]:
# Load and prepare the data
def load_data():
    # Load movie ratings data
    # In a real-world scenario, you'd load this from a file or database
    ratings_data = {
        'user_id': [1, 1, 1, 2, 2, 3, 3, 3, 4, 4],
        'movie_id': [1, 2, 3, 1, 2, 1, 3, 4, 2, 4],
        'rating': [5, 4, 3, 3, 2, 4, 5, 4, 3, 5]
    }
    df = pd.DataFrame(ratings_data)
    return df

In [3]:
# Create a user-item matrix
def create_matrix(df):
    user_item_matrix = df.pivot(index='user_id', columns='movie_id', values='rating')
    return user_item_matrix


In [4]:
# Compute user similarity
def compute_similarity(matrix):
    # Fill NaN values with 0
    imputer = SimpleImputer(strategy='constant', fill_value=0)
    matrix_imputed = pd.DataFrame(imputer.fit_transform(matrix), columns=matrix.columns, index=matrix.index)

    # Compute cosine similarity
    user_similarity = cosine_similarity(matrix_imputed)
    user_similarity_df = pd.DataFrame(user_similarity, index=matrix.index, columns=matrix.index)
    return user_similarity_df


In [5]:
# Get top N similar users
def get_similar_users(user_id, user_similarity_df, n=2):
    similar_users = user_similarity_df[user_id].sort_values(ascending=False)[1:n+1]
    return similar_users


In [6]:
# Generate recommendations
def generate_recommendations(user_id, matrix, user_similarity_df, n=2):
    similar_users = get_similar_users(user_id, user_similarity_df, n)
    user_movies = set(matrix.loc[user_id].dropna().index)

    recommendations = {}
    for similar_user in similar_users.index:
        similar_user_movies = set(matrix.loc[similar_user].dropna().index)
        new_movies = similar_user_movies - user_movies
        for movie in new_movies:
            if movie not in recommendations:
                recommendations[movie] = matrix.loc[similar_user, movie]

    sorted_recommendations = sorted(recommendations.items(), key=lambda x: x[1], reverse=True)
    return sorted_recommendations


In [7]:
# Main function
def main():
    # Load data
    df = load_data()
    print("Data loaded successfully.")

    # Create user-item matrix
    user_item_matrix = create_matrix(df)
    print("\nUser-Item Matrix:")
    print(user_item_matrix)

    # Compute user similarity
    user_similarity_df = compute_similarity(user_item_matrix)
    print("\nUser Similarity Matrix:")
    print(user_similarity_df)

    # Generate recommendations for a specific user
    user_id = 1
    recommendations = generate_recommendations(user_id, user_item_matrix, user_similarity_df)

    print(f"\nTop recommendations for User {user_id}:")
    for movie, rating in recommendations:
        print(f"Movie {movie}: Predicted rating {rating:.2f}")

if __name__ == "__main__":
    main()

Data loaded successfully.

User-Item Matrix:
movie_id    1    2    3    4
user_id                     
1         5.0  4.0  3.0  NaN
2         3.0  2.0  NaN  NaN
3         4.0  NaN  5.0  4.0
4         NaN  3.0  NaN  5.0

User Similarity Matrix:
user_id         1         2         3         4
user_id                                        
1        1.000000  0.902134  0.655610  0.291043
2        0.902134  1.000000  0.440831  0.285391
3        0.655610  0.440831  1.000000  0.454311
4        0.291043  0.285391  0.454311  1.000000

Top recommendations for User 1:
Movie 4: Predicted rating 4.00
